In [1]:
# 1. Install Transformers (if not already installed)
!pip install transformers

In [ ]:
!pip install transformers datasets accelerate peft -q


#Importing required libraries

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("stanford-crfm/biomedlm")

model = AutoModelForCausalLM.from_pretrained(
    "stanford-crfm/biomedlm",
    torch_dtype=torch.float16,
    device_map="auto"
).to("cuda" if torch.cuda.is_available() else "cpu")


#Inference

In [ ]:
# Example input
prompt = "A 30-year-old woman presents with fever, chills, and dysuria. What is the appropriate treatment?\n\nClinician response:"

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=150,
        temperature=0.8,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id  # Prevents warning
    )

# Decode
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


In [13]:
!pip install transformers accelerate peft

In [2]:
import pandas as pd
import numpy as np

print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)


Pandas version: 2.1.4
Numpy version: 1.26.4


In [3]:
import pandas as pd
import numpy as np

In [4]:
import os
print(os.path.exists("/kaggle/input/clinical-reasoning-dataset/train.csv"))


True


In [5]:

# Load the CSV file
df = pd.read_csv("/kaggle/input/clinical-reasoning-dataset/train.csv")

In [7]:
from datasets import Dataset
df = df[['Prompt', 'Clinician']].dropna()

# Rename columns for Hugging Face
# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df.rename(columns={'Prompt': 'input', 'Clinician': 'output'}))


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

2025-06-16 23:52:29.145229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750117949.362101     178 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750117949.421223     178 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

def preprocess(example):
    return tokenizer(
        "clinical prompt: " + example["input"],
        text_target=example["output"],
        max_length=512,
        padding="max_length",
        truncation=True
    )

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

TypeError: can only concatenate str (not "list") to str

In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

# Add padding token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def preprocess(examples):
    """
    Correct preprocessing function for T5 fine-tuning
    """
    # Handle both single examples and batches
    if isinstance(examples["input"], str):
        # Single example
        inputs = ["clinical prompt: " + examples["input"]]
        targets = [examples["output"]]
    else:
        # Batch of examples
        inputs = ["clinical prompt: " + inp for inp in examples["input"]]
        targets = examples["output"]
    
    # Tokenize inputs
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt" if isinstance(examples["input"], str) else None
    )
    
    # Tokenize targets using the target tokenizer context
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt" if isinstance(examples["input"], str) else None
        )
    
    # Replace padding token ids with -100 so they're ignored in loss computation
    labels_ids = labels["input_ids"]
    if isinstance(labels_ids, list):
        labels_ids = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels_ids
        ]
    else:
        # Single example
        labels_ids = [(l if l != tokenizer.pad_token_id else -100) for l in labels_ids[0]]
    
    model_inputs["labels"] = labels_ids
    return model_inputs

# Apply preprocessing
print("Preprocessing dataset...")
tokenized_dataset = dataset.map(
    preprocess, 
    batched=True,
    remove_columns=dataset["train"].column_names  # Remove original columns
)

print("Preprocessing completed!")
print(f"Train dataset size: {len(tokenized_dataset['train'])}")
print(f"Validation dataset size: {len(tokenized_dataset['validation'])}")

# Check the first processed example
print("\nSample processed data:")
print(f"Input IDs shape: {len(tokenized_dataset['train'][0]['input_ids'])}")
print(f"Labels shape: {len(tokenized_dataset['train'][0]['labels'])}")
print(f"Attention mask shape: {len(tokenized_dataset['train'][0]['attention_mask'])}")

Preprocessing dataset...


KeyError: "Column train not in the dataset. Current columns in the dataset: ['input', 'output']"